<a href="https://colab.research.google.com/github/eyupdalan/BLM6114-project/blob/main/BLM6114_project_google_gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kütüphaneler

In [1]:
!pip install -q torch accelerate transformers datasets huggingface-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.8 MB/s eta 0:00:00


## Modellere göre sınıflandırma (Çözebilir mi?)

Modeller:

https://huggingface.co/google/gemma-2-9b-it


### Modelin yüklenmesi

In [4]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import notebook_login
from google.colab import userdata
from huggingface_hub import login

token = userdata.get('HF_TOKEN')
login(token=token)

# pipeline'ları oluşturma
google_gemma = pipeline(
    "text-generation",
    model="google/gemma-2-9b-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",  # replace with "mps" to run on a Mac device
)


config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Device set to use cuda


### Değerlendirme fonksiyonu

In [7]:
import re

def extract_number(text):
    # Metindeki sayıları bul, sonuncusunu float’a çevir. Son sayı cevap.
    nums = re.findall(r"[-+]?\d+\.?\d*", text.replace(",","."))
    return float(nums[-1]) if nums else None

def predict_gg_gemma(question):
  prompt = (
    f"Soru: {question}\n"
    "Soruya kısa ve net bir yanıt ver, adım adım çözme, yanıtlarında ‘24 \times \frac’ gibi notasyonlar kullanma, bunu da ‘Sonuç: <değer>’ formatında göster."
  )
  messages = [
    {"role": "user", "content": prompt},
  ]
  outputs = google_gemma(messages, max_new_tokens=256)
  assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
  return assistant_response


def label_prediction(question, true_answer,):
  gg_gemma_result = predict_gg_gemma(question)
  print("---------------------------------------")
  print(f"gg_gemma_result: {gg_gemma_result}")
  print("---------------------------------------")

  gg_gemma_prediction = extract_number(gg_gemma_result)
  print("---------------------------------------")
  print(f"gg_gemma_prediction: {gg_gemma_prediction}")
  print("---------------------------------------")

  real = extract_number(true_answer)
  # Tahmin gerçek değerle eşleşiyorsa 1, yoksa 0
  is_gg_gemma_correct = 1 if (gg_gemma_prediction is not None and real is not None and abs(gg_gemma_prediction-real)<1e-3) else 0
  return gg_gemma_result, is_gg_gemma_correct

### Örnek Soru-Cevap Karşılaştırması

In [8]:
import os
os.environ["TRANSFORMERS_VERBOSITY"] = "info"

df = pd.read_csv("gsm8k_tr_classified.csv")

results = []

# Sadece ilk 5 kayıt için döngü
for i in range(5):
  q = df.question.iloc[i]
  true_ans = df.answer.iloc[i]

  gg_gemma_result, is_gg_gemma_correct = label_prediction(q, true_ans)

  # DataFrame'e yeni sütunlar ekleyelim
  df.loc[i, "gg_gemma_result"] = gg_gemma_result
  df.loc[i, "is_gg_gemma_correct"] = is_gg_gemma_correct

# Sonuçları DataFrame olarak görüntüle (İlk 5 satır için)
display(df.head())

/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


---------------------------------------
gg_gemma_result: Sonuç: 270
---------------------------------------
---------------------------------------
gg_gemma_prediction: 270.0
---------------------------------------
---------------------------------------
gg_gemma_result: İki düzine 24'tür. Katherine 24 yaşındayken Mel 24 - 3 = 21 yaşındadır. 

Sonuç: 21
---------------------------------------
---------------------------------------
gg_gemma_prediction: 21.0
---------------------------------------
---------------------------------------
gg_gemma_result: Sonuç: 24
---------------------------------------
---------------------------------------
gg_gemma_prediction: 24.0
---------------------------------------
---------------------------------------
gg_gemma_result: Kyle başlangıçta 2 şişe * 15 yıldız/şişe = 30 yıldız alır.
Kyle daha sonra 3 şişe * 15 yıldız/şişe = 45 yıldız daha alır.
Kyle toplamda 30 yıldız + 45 yıldız = 75 yıldız yapmalıdır.

Sonuç: 75
-----------------------------------

,question,answer,question_type,solution_method,gg_gemma_result,is_gg_gemma_correct
0,Borris tekel bayisi her 6 ayda bir 90 kilogram...,Borris şu anda her 6 ayda 90 kilogram üzüm kul...,oran-orantı,yüzde hesaplama,Sonuç: 270,0.0
1,"Mel, Katherine'den üç yaş küçük. Katherine ik...",Katherine iki düzine yaşına geldiğinde 24 yaşı...,yaş problemi,doğrudan çıkarma,İki düzine 24'tür. Katherine 24 yaşındayken Me...,1.0
2,James 2 ağacındaki tüm meyveleri toplar. Her ...,James 24 ağaç dikmiştir.,oran-orantı,birim oran,Sonuç: 24,1.0
3,"Kyle, her biri 15 origami yıldızı alabilen 2 c...",Kyle toplamda 5 cam şişe satın aldı (2 + 3). H...,çarpma,doğrudan çarpma,Kyle başlangıçta 2 şişe * 15 yıldız/şişe = 30 ...,1.0
4,Mark'ın iki evcil hayvanı var: Saniyede 10 adı...,Kaplumbağanın yarışı berabere bitirmesi için 1...,oran-orantı,birim oran,Sonuç: 12,0.0
